In [1]:
import pandas as pd
import numpy as np
import pickle
import copy
import nltk

from wordcloud import WordCloud
from matplotlib.colors import ListedColormap
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from bertopic import BERTopic


/home/cattiaux/anaconda3/envs/wassati/lib/python3.9/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/cattiaux/anaconda3/envs/wassati/lib/python3.9/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/cattiaux/anaconda3/envs/wassati/lib/python3.9/site-packages/

In [2]:
df = pd.read_csv("/media/cattiaux/DATA/Wassati/team_data/schneider/df_all_labelled.csv", dtype={'year': str})

/tmp/ipykernel_6144/1730781217.py:1: DtypeWarning: Columns (10,11,15,19,20,21,23,34,35,42,43,44,45) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/media/cattiaux/DATA/Wassati/team_data/schneider/df_all_labelled.csv", dtype={'year': str})


In [3]:
### chargement du model bertopic

def load_bertopic_model(filename):
    """
    Load a BERTopic model and associated data from a file.
    
    :param filename: The name of the file to load the data from.
    :return: A tuple containing the loaded BERTopic model, topics, probs, and docs variables.
    """
    # Load the BERTopic model
    topic_model = BERTopic.load(filename)
    
    # Load the topics, probs, and docs variables
    with open(filename + '_data.pkl', 'rb') as f:
        topics, probs, embeddings, docs = pickle.load(f)
    
    return topic_model, topics, probs, embeddings, docs

topic_model, topics, probs, embeddings, docs = load_bertopic_model('../models/raw_keybert_bertopic_model')

def create_merged_model(docs, bertopic_model, topics_to_merge_dict, label_names_dict):
    """
    Create a new BERTopic model by merging topics from an existing model.

    This function takes as input a list of documents `docs`, an existing BERTopic model `bertopic_model`, a dictionary `topics_to_merge_dict` specifying which topics to merge, and a dictionary `label_names_dict` specifying the labels for the merged topics.

    The function creates a deep copy of the input BERTopic model and merges the specified topics using the `merge_topics` method. Then, it sets the topic labels for the merged model using the `set_topic_labels` method and the provided `label_names_dict`.

    The resulting merged BERTopic model is then returned.

    Parameters:
        docs (list): A list of documents used to fit the BERTopic model.
        bertopic_model (BERTopic): The input BERTopic model to be merged.
        topics_to_merge_dict (dict): A dictionary specifying which topics to merge. The keys are the topic numbers to be merged, and the values are the topic numbers into which they should be merged.
        label_names_dict (dict): A dictionary specifying the labels for the merged topics. The keys are the topic numbers, and the values are the corresponding labels.

    Returns:
        BERTopic: The resulting merged BERTopic model.
    """
    topic_model_merged = copy.deepcopy(bertopic_model)
    topic_model_merged.merge_topics(docs, topics_to_merge_dict)

    # Create a dictionary to match the aggregated name to their corresponding topic number
    mergedtopic_labels_dict = {i-1: item for i, item in enumerate(label_names_dict)}
    # Set topic labels for the aggregated model
    topic_model_merged.set_topic_labels(mergedtopic_labels_dict)

    return topic_model_merged### Création du modèle bertopic aggrégé pour topics finaux

# List of topics numbers. Each value of this list is a list that contains the topic number of the topics to join together
topics_to_merge = [ [42,3,0,13], #Delivery Deadlines : challenges and strategies involved in managing delivery deadlines in logistics operations. (vert)
                    [20,50,27], #Quotation and Pricing Strategies (vert bas)
                    [35,32], #Touch Panels and Screens (rouge, haut)
                    [40,36], #Frequency Converters : frequency converters used in industrial applications and the technical support provided by manufacturers and suppliers (rouge, suite)
                    [37,21,6,12,9,4,1,14,16,31,19], #“Automation Components” : hardware and software components used in industrial automation systems. (rouge centre)
                    [33,46,8], #Product Evaluation : evaluate the quality, affordability and reliability of products and services (rouge, fin)
                    [44,51,23,41,49,57,22], #Customer Support : Reliability and Quality in Customer Service and Support (bleu ciel)
                    [58,59], #Quick Customer Service (marron)
                    [38,10,26,52,39,43], #Problem Solving and Communication (focus on the importance of being efficient and precise when solving problems) (jaune)
                    [45,47,55,53,54], #Assistance and Guidance (noir)
                    [29,30,11,24], #Power Supply Issues (2e vert, haut)
                    [7,5,2,25,15,34,18,28,17], #Technical Support (2e vert, bas)
                    [48,56] #None : positive feedback (2e rouge)
]

# Set the topic names for the new aggregated topic
# It must match the order from the topics_to_merge list
label_names = [
    "Outliers",
    "Automation Components",
    "Technical Support",
    "Delivery Deadlines",
    "Problem Solving & Comm",
    "Power Supply Issues",
    "Customer Support", #Reliability and Quality in Customer Service and Support
    "Product Evaluation",
    "Pricing", #Quotation and Pricing Strategies
    "Assistance", #Assistance and Guidance
    "Touch Screens", #Touch Panels and Screens
    "Frequency Converters",
    "Positive feedback",
    "Quick Customer Service"
    ]

# Create a new merged bertopic model 
topic_model_merged = create_merged_model(docs, topic_model, topics_to_merge, label_names)
# topic_model_merged.get_topic_info()

/home/cattiaux/anaconda3/envs/wassati/lib/python3.9/site-packages/bertopic/vectorizers/_ctfidf.py:69: RuntimeWarning: divide by zero encountered in divide
  idf = np.log((avg_nr_samples / df)+1)


In [ ]:
### Wordcloud
### Wordcloud 
# Lemmatize keywords and recompute their probabilities for the wordcloud image

# Download required resources
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('punkt')

def lemmatize_words(topic_words):
    """
    Lemmatize the words and combine their probabilities.

    This function takes as input a list of tuples `topic_words`, where each tuple contains a word and its probability. The function lemmatizes each word using the WordNetLemmatizer from the NLTK library, and combines the probabilities of the lemmas and their inflected forms.

    The resulting dictionary, where the keys are the lemmas and the values are their combined probabilities, is then returned.

    Parameters:
        topic_words (list): A list of tuples, where each tuple contains a word (str) and its probability (float).

    Returns:
        dict: A dictionary where the keys are the lemmas (str) and the values are their combined probabilities (float).
    """
    def get_wordnet_pos(treebank_tag):
        """
        Convert NLTK part of speech tags to WordNet tags.

        This function takes as input a part of speech tag in the format used by the NLTK library and returns the corresponding WordNet tag. The mapping between NLTK and WordNet tags is as follows:
        - 'J' (adjective) maps to `wordnet.ADJ`
        - 'V' (verb) maps to `wordnet.VERB`
        - 'N' (noun) maps to `wordnet.NOUN`
        - 'R' (adverb) maps to `wordnet.ADV`
        - All other tags map to `wordnet.NOUN`

        Parameters:
            treebank_tag (str): The NLTK part of speech tag to be converted.

        Returns:
            str: The corresponding WordNet part of speech tag.
        """
        if treebank_tag.startswith('J'):
            return wordnet.ADJ
        elif treebank_tag.startswith('V'):
            return wordnet.VERB
        elif treebank_tag.startswith('N'):
            return wordnet.NOUN
        elif treebank_tag.startswith('R'):
            return wordnet.ADV
        else:
            return wordnet.NOUN

    # Create a lemmatizer object
    lemmatizer = WordNetLemmatizer()
    # Create a dictionary to store the lemmas and their probabilities
    lemma_prob = {}

    # Lemmatize each word and combine their probabilities
    for word, prob in topic_words:
        # Tokenize the word and get its part of speech
        tokens = nltk.word_tokenize(word)
        pos = nltk.pos_tag(tokens)[0][1]
        # Get the WordNet part of speech tag
        wordnet_pos = get_wordnet_pos(pos)
        # Lemmatize the word
        lemma = lemmatizer.lemmatize(word, pos=wordnet_pos)
        
        # Combine the probabilities of the lemma and its inflected forms
        if lemma in lemma_prob:
            lemma_prob[lemma] += prob
        else:
            lemma_prob[lemma] = prob
    
    return lemma_prob

def recalculate_probabilities(lemma_prob, docs, topic_model):
    """
    Recalculate the c-TF-IDF scores for the lemmas.

    This function takes as input a dictionary `lemma_prob` containing the lemmas and their probabilities, a list of documents `docs`, and a BERTopic model `topic_model`. The function recalculates the c-TF-IDF scores for the lemmas using the provided documents and BERTopic model.

    The function first calculates the term frequencies for each lemma using the CountVectorizer from the BERTopic model. Then, it calculates the inverse document frequencies for each lemma and uses these values to compute the c-TF-IDF scores. The c-TF-IDF scores are then normalized and used to update the probabilities of the lemmas.

    The resulting dictionary, where the keys are the lemmas and the values are their updated probabilities, is then returned.

    Parameters:
        lemma_prob (dict): A dictionary where the keys are the lemmas (str) and the values are their probabilities (float).
        docs (list): A list of documents used to fit the BERTopic model.
        topic_model (BERTopic): The BERTopic model used to calculate the c-TF-IDF scores.

    Returns:
        dict: A dictionary where the keys are the lemmas (str) and the values are their updated probabilities (float).
    """
    # Calculate the term frequencies using the provided CountVectorizer
    X = topic_model.vectorizer_model.transform(docs)
    
    # Calculate the term frequencies for each lemma
    tf = {}
    for lemma, prob in lemma_prob.items():
        index = topic_model.vectorizer_model.vocabulary_.get(lemma)
        if index is not None:
            tf[lemma] = np.sum(X[:, index])

    # Calculate the inverse document frequencies for each lemma
    df = np.sum(X > 0, axis=0)
    N = X.shape[0]
    idf = np.log(N / (df + 1))

    # Calculate the c-TF-IDF scores for each lemma and normalize them
    c_tf_idf = {}
    for lemma, prob in lemma_prob.items():
        index = topic_model.vectorizer_model.vocabulary_.get(lemma)
        if index is not None:
            c_tf_idf[lemma] = tf[lemma] * idf[0, index]
    c_tf_idf_sum = np.sum(list(c_tf_idf.values()))
    for lemma, score in c_tf_idf.items():
        c_tf_idf[lemma] /= c_tf_idf_sum

    # Update the probabilities of the lemmas based on their c-TF-IDF scores
    new_lemma_prob = {}
    for lemma, prob in lemma_prob.items():
        if lemma in c_tf_idf:
            new_lemma_prob[lemma] = c_tf_idf[lemma]
    
    return new_lemma_prob

def get_topic_words(bertopic_model, topic, top_n=10):
    """
    Get the top n words for a given topic.

    This function takes as input a BERTopic model `topic_model`, a topic number `topic`, and an optional integer parameter `top_n` specifying the number of words to return. The function returns the top n words for the given topic, along with their probabilities, as a list of tuples.

    The function first retrieves the c-TF-IDF matrix and feature names from the BERTopic model. Then, it gets the row of the c-TF-IDF matrix corresponding to the given topic and uses it to find the indices of the top n words. Finally, it retrieves the words and their probabilities and returns them as a list of tuples.

    Parameters:
        topic_model (BERTopic): The BERTopic model used to calculate the topic words.
        topic (int): The topic number for which to retrieve the top n words.
        top_n (int): An optional integer parameter specifying the number of words to return. Defaults to 10.

    Returns:
        list: A list of tuples, where each tuple contains a word (str) and its probability (float).
    """
    # Get the c-TF-IDF matrix and feature names
    c_tf_idf = bertopic_model.c_tf_idf_.toarray()
    feature_names = bertopic_model.vectorizer_model.get_feature_names_out()
    
    # Get the row of the c-TF-IDF matrix corresponding to the topic
    topic_row = c_tf_idf[topic]
    
    # Get the indices of the top_n words for the topic
    top_n_indices = np.argsort(topic_row)[::-1][:top_n]
    
    # Get the words and their probabilities
    words = [feature_names[i] for i in top_n_indices]
    probabilities = [topic_row[i] for i in top_n_indices]
    
    # Return the words and their probabilities as a list of tuples
    return list(zip(words, probabilities))

def group_docs_by_topic(docs, bertopic_model):
    """
    Group documents by their assigned topic.

    This function takes as input a list of documents `docs` and a BERTopic model `bertopic_model`. It creates a dictionary where the keys are topic numbers and the values are lists of documents assigned to each topic.

    Parameters:
        docs (list): A list of documents used to fit the BERTopic model.
        bertopic_model (BERTopic): The BERTopic model used to assign topics to the documents.

    Returns:
        dict: A dictionary where the keys are topic numbers (int) and the values are lists of documents (list) assigned to each topic.
    """
    docs_by_topic = {}
    for doc, topic in zip(docs, bertopic_model.topics_):
        if topic+1 not in docs_by_topic:
            docs_by_topic[topic+1] = []
        docs_by_topic[topic+1].append(doc)
    
    return docs_by_topic 

def get_word_freq(bertopic_model, docs, topic, top_n=10, scale=1, lemmatize=False):
    """
    Get the word frequencies for a given topic.

    This function takes as input a BERTopic model `bertopic_model`, a list of documents `docs`, a topic number `topic`, an optional integer parameter `top_n` specifying the number of words to include, an optional float parameter `scale` used to scale the probabilities of the words and an optional boolean parameter `lemmatize` which determines whether to lemmatize the words before calculating their frequencies.

    The function first retrieves the top n words for the given topic using the `get_topic_words` function and scales their probabilities using the provided `scale` parameter. If `lemmatize` is `True`, the function lemmatizes the words using the `lemmatize_words` function and recalculates their probabilities using the `recalculate_probabilities` function. Otherwise, it uses the original words and their probabilities.

    Parameters:
        bertopic_model (BERTopic): The BERTopic model used to calculate the topic words.
        docs (list): A list of documents used to fit the BERTopic model.
        topic (int): The topic number for which to calculate the word frequencies.
        top_n (int): An optional integer parameter specifying the number of words to include. Defaults to 10.
        scale (float): An optional float parameter used to scale the probabilities of the words. Defaults to 1.
        lemmatize (bool): An optional boolean parameter used to determine whether to lemmatize the words before calculating their frequencies. Defaults to False.

    Returns:
        dict: A dictionary where the keys are the words/lemmas (str) and the values are their probabilities (float).
    """
    # Get the topic words and their probabilities
    topic_words = get_topic_words(bertopic_model, topic, top_n=top_n)
    # Scale the probabilities
    topic_words = [(word, prob ** scale) for word, prob in topic_words]

    if lemmatize:
        # Group documents by their assigned topic.
        docs_by_topic = group_docs_by_topic(docs, bertopic_model)
        # get the documents assigned to a specific topic
        my_docs = docs_by_topic.get(topic, [])
        # Lemmatize the words and combine their probabilities
        lemma_prob = lemmatize_words(topic_words)
        # Recalculate the c-TF-IDF scores for the lemmas
        topic_words_lemma = recalculate_probabilities(lemma_prob, my_docs, bertopic_model)
        # Create a dictionary with the lemmas and their probabilities
        word_freq = {lemma: prob for lemma, prob in topic_words_lemma.items()}
    
    else:
        # Create a dictionary with the words and their probabilities
        word_freq = {word: prob for word, prob in topic_words}
    
    return word_freq 

def create_wordcloud(bertopic_model, docs, topic, top_n=10, scale=1, lemmatize=False, stopwords=None, wordcloud_kwargs=None):
    """
    Create a word cloud from a BERTopic model and a topic.

    This function takes as input a BERTopic model `topic_model`, a list of documents `docs`, a topic number `topic`, an optional integer parameter `top_n` specifying the number of words to include in the word cloud, an optional float parameter `scale` used to scale the probabilities of the words, an optional boolean parameter `lemmatize` which determines whether to lemmatize the words before creating the word cloud, an optional list of stopwords `stopwords` to be removed from the word cloud, and an optional dictionary of keyword arguments `wordcloud_kwargs` to be passed to the WordCloud constructor.

    The function first retrieves the top n words for the given topic using the `get_topic_words` function and scales their probabilities using the provided `scale` parameter. If `lemmatize` is `True`, the function lemmatizes the words using the `lemmatize_words` function and recalculates their probabilities using the `recalculate_probabilities` function. Otherwise, it uses the original words and their probabilities.

    The function then removes any stopwords from the list of words (if provided) and creates a word cloud using the WordCloud class from the wordcloud library. The resulting word cloud is then returned.

    Parameters:
        topic_model (BERTopic): The BERTopic model used to calculate the topic words.
        docs (list): A list of documents used to fit the BERTopic model.
        topic (int): The topic number for which to create the word cloud.
        top_n (int): An optional integer parameter specifying the number of words to include in the word cloud. Defaults to 10.
        scale (float): An optional float parameter used to scale the probabilities of the words. Defaults to 1.
        lemmatize (bool): An optional boolean parameter used to determine whether to lemmatize the words before creating the word cloud. Defaults to False.
        stopwords (list): An optional list of stopwords to be removed from the word cloud. Defaults to None.
        wordcloud_kwargs (dict): An optional dictionary of keyword arguments to be passed to the WordCloud constructor. Defaults to None.

    Returns:
        wordcloud.WordCloud: The resulting word cloud.
    """
    # Get the word frequencies for a given topic.
    word_freq = get_word_freq(bertopic_model, docs, topic, top_n=top_n, scale=scale, lemmatize=lemmatize)

    # Remove stopwords from word_freq if provided
    if stopwords:
        word_freq = {word: freq for word, freq in word_freq.items() if word not in stopwords}
    
    # Create the word cloud using the words/lemmas and their probabilities
    wc = WordCloud(**(wordcloud_kwargs or {}))
    wc.generate_from_frequencies(word_freq)

    # Display the word cloud
    # plt.imshow(wc, interpolation='bilinear')
    # plt.axis("off")
    # plt.show()
    return wc

def create_wordclouds_bertopic(bertopic_model, docs, top_n=10, scale=1, lemmatize=False, stopwords=None, wordcloud_kwargs=None, to_save=False, save_path=None):
    """
    Create word clouds for all topics in a BERTopic model.

    This function takes as input a BERTopic model `bertopic_model`, a list of documents `docs`, an optional integer parameter `top_n` specifying the number of words to include in each word cloud, an optional float parameter `scale` used to scale the probabilities of the words, an optional boolean parameter `lemmatize` which determines whether to lemmatize the words before creating the word clouds, an optional list of stopwords `stopwords` to be removed from the word clouds, an optional dictionary of keyword arguments `wordcloud_kwargs` to be passed to the WordCloud constructor, an optional boolean parameter `to_save` which determines whether to save the word clouds as image files, and an optional string parameter `save_path` specifying the path where the image files should be saved.

    The function first retrieves the topic information from the BERTopic model and sets the index of the resulting DataFrame to be the topic number. Then, it loops over the topic numbers in the DataFrame and calls the `create_wordcloud` function to create a word cloud for each topic. If `to_save` is `True`, it saves each word cloud as an image file at the specified location using the custom name of the topic.

    The resulting dictionary, where the keys are the custom names of the topics and the values are their corresponding word clouds, is then returned.

    Parameters:
        bertopic_model (BERTopic): The BERTopic model used to calculate the topic words.
        docs (list): A list of documents used to fit the BERTopic model.
        top_n (int): An optional integer parameter specifying the number of words to include in each word cloud. Defaults to 10.
        scale (float): An optional float parameter used to scale the probabilities of the words. Defaults to 1.
        lemmatize (bool): An optional boolean parameter used to determine whether to lemmatize the words before creating the word clouds. Defaults to False.
        stopwords (list): An optional list of stopwords to be removed from each word cloud. Defaults to None.
        wordcloud_kwargs (dict): An optional dictionary of keyword arguments to be passed to each WordCloud constructor. Defaults to None.
        to_save (bool): An optional boolean parameter used to determine whether to save each word cloud as an image file. Defaults to False.
        save_path (str): An optional string parameter specifying the path where each image file should be saved. Only used if `to_save` is True. Defaults to None.

    Returns:
        dict: A dictionary where the keys are the custom names of the topics (str) and the values are their corresponding word clouds (wordcloud.WordCloud).
    """
    # Check that save_path is provided if to_save is True
    if to_save and save_path is None:
        raise ValueError("If to_save is True, save_path must be provided")
    
    # Get the topic information
    topic_info = bertopic_model.get_topic_info()
    # Set the index of the DataFrame to be the topic number
    topic_info = topic_info.set_index('Topic')
    wc_pics={}
    # Loop over the topic numbers in the DataFrame
    for topic_number in topic_info.index:
        # Get the custom name of the current topic for saving purpose
        topic_custom_name = topic_info.loc[topic_number, 'CustomName']
        wc_pic = create_wordcloud(bertopic_model, docs, topic_number+1, top_n=top_n, scale=scale, lemmatize=lemmatize, stopwords=stopwords, wordcloud_kwargs=wordcloud_kwargs)
        wc_pics.update({topic_custom_name: wc_pic})

        if to_save and lemmatize:
            wc_pic.to_file(f'{save_path}/{topic_custom_name}_lemmatized.png')
        elif to_save and lemmatize==False:
            wc_pic.to_file(f'{save_path}/{topic_custom_name}.png')

    return wc_pics

# examples of default colors for the wordcloud
# colormaps = [
#     'viridis', 'plasma', 'inferno', 'magma', 'cividis',
#     'Greys', 'Purples', 'Blues', 'Greens', 'Oranges',
#     'Reds', 'YlOrBr', 'YlOrRd', 'OrRd', 'PuRd',
#     'RdPu', 'BuPu', 'GnBu', 'PuBu', 'YlGnBu',
#     'PuBuGn', 'BuGn', 'YlGn'
# ]

# Create a custom colormap
# colors = ["#294D61", "#6DA5C0", "#0F969C", "#0C7075", "#072E33","#05161A"]
colors = ["#F1916D", "#F5D7DB", "#BD83B8", "#473E66", "#1B3358","#06142E"]
my_cmap = ListedColormap(colors)

# Create a binary mask from an image
# make sure that the image is black and white, where black pixels indicate where to draw words and white pixels indicate where not to draw words.
# mask = np.array(Image.open('circle.png'))

wordcloud_stopwords = ["get","would","us","good","like","goods","got","able","quite","always","nothing","add","everything","think","gave","due","find","say","took","still","within","22","10","one","makhloufi","saliger","kessler","naldrin","hidde","mohua","possible","since","could","especially","altivar","every","anyway","egawa","sometimes"]

wordcloud_kwargs = {
    'width': 800,
    'height': 400,
    'background_color': 'black',
    'contour_width': 100,
    'contour_color': 'red',
    'colormap': 'GnBu',
    "contour_width": 5,
    "contour_color": 'red'
    # "mask":mask
}

wordclouds = create_wordclouds_bertopic(topic_model_merged, docs, 
                           top_n=50, scale=1, 
                           lemmatize=True, 
                           stopwords=wordcloud_stopwords, 
                           wordcloud_kwargs=wordcloud_kwargs,
                           to_save=True, 
                           save_path="../data/wordclouds/test2")